In [1]:
from __future__ import division
import gym
import torch
import random
import numpy as np
import torch
from PIL import Image
import torch
import torch.nn as nn
from collections import namedtuple
from collections import deque
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
from matplotlib.pyplot import imshow
from PIL import Image
from wrappers import make_atari, wrap_deepmind, wrap_pytorch
import queue
from torch import optim
import matplotlib.pyplot as plt
import math
import pandas as pd 
import os 
import pickle 

In [2]:
class QNet(torch.nn.Module):
    def __init__(self,obs_shape,act_shape):
        super(QNet, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(7*7*64,512)
        self.fc2 = nn.Linear(512,1)
        self.fc3 = nn.Linear(7*7*64,512)
        self.fc4 = nn.Linear(512,act_shape)
        
    def forward(self, x):
        #Conv
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = x.view(x.shape[0],-1)
        
        #Fc
        x1 = x
        x = self.fc1(x)
        x = self.relu(x)
        v = self.fc2(x)
        
        x1 = self.fc3(x1)
        x1 = self.relu(x1)
        adv = self.fc4(x1)
        
        q_s_a = v + adv - adv.mean()
        
        return q_s_a

In [3]:
def eps_greedy(epsilon,state,net):
    if(np.random.random()<epsilon):
        action = np.random.randint(ACT_SHAPE)
    else:
        qvalues = net(state)
        action = torch.argmax(qvalues).item()
    return action    

In [4]:
env    = make_atari('PongNoFrameskip-v4')
env    = wrap_deepmind(env)
env    = wrap_pytorch(env)

In [5]:
def test(net,evaluation_episodes):
    state = env.reset()
    net.eval()
    state = torch.Tensor(state).cuda()
    state = state.unsqueeze(0)
    rewards = []
    count = 0
    episode_reward = 0
    while(True):
        action = eps_greedy(0,state,net)  
        next_state, reward, done,info = env.step(action)
        next_state = torch.Tensor(next_state).cuda().unsqueeze(0)
        state = next_state
        episode_reward += reward
        if done:
            count += 1
            print('Episode ',count,end=' ')
            print('Reward ',episode_reward)
            rewards.append(episode_reward)
            state = env.reset()
            state = torch.Tensor(state).cuda()
            state = state.unsqueeze(0)
            episode_reward = 0
        if(count == evaluation_episodes):
            break
    env.close()


    return sum(rewards)/len(rewards)

In [6]:
net = QNet(env.observation_space.shape,env.action_space.n).cuda()
net.load_state_dict(torch.load('./dueling-logs/dueling-model2000000.pth'))
test(net,50)

Episode  1 Reward  21.0
Episode  2 Reward  20.0
Episode  3 Reward  21.0
Episode  4 Reward  20.0
Episode  5 Reward  21.0
Episode  6 Reward  21.0
Episode  7 Reward  21.0
Episode  8 Reward  21.0
Episode  9 Reward  19.0
Episode  10 Reward  21.0
Episode  11 Reward  21.0
Episode  12 Reward  20.0
Episode  13 Reward  20.0
Episode  14 Reward  21.0
Episode  15 Reward  21.0
Episode  16 Reward  21.0
Episode  17 Reward  20.0
Episode  18 Reward  21.0
Episode  19 Reward  21.0
Episode  20 Reward  21.0
Episode  21 Reward  21.0
Episode  22 Reward  21.0
Episode  23 Reward  19.0
Episode  24 Reward  21.0
Episode  25 Reward  20.0
Episode  26 Reward  21.0
Episode  27 Reward  21.0
Episode  28 Reward  21.0
Episode  29 Reward  19.0
Episode  30 Reward  19.0
Episode  31 Reward  21.0
Episode  32 Reward  21.0
Episode  33 Reward  20.0
Episode  34 Reward  20.0
Episode  35 Reward  21.0
Episode  36 Reward  19.0
Episode  37 Reward  20.0
Episode  38 Reward  20.0
Episode  39 Reward  19.0
Episode  40 Reward  19.0
Episode  

20.4